In [ ]:
!pip install einops
from diffusion import VPSDE
import torchvision
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from diffusion import VPSDE
from torch.utils.data import DataLoader
from torch.optim import Adam
from guided_diffusion import  train_classification_network_mnist, Net

In [ ]:
device = 'cuda'
data = 'MNIST'
model = Net().to(device)
sde = VPSDE(100, 0.1, 20, device = device)

''' hyper params'''
n_epochs =   50
batch_size =  32
lr=1e-4
optimizer = Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
tfm = transforms.Compose([transforms.ToTensor(), transforms.Pad(2)])
data = torchvision.datasets.MNIST(f'./data/', transform=tfm, download = True)
data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
train_classification_network_mnist(data_loader, model, sde, optimizer, epochs=100)

Epoch: 0 and Loss: 3821.132080078125
Epoch: 1 and Loss: 3817.44140625
Epoch: 2 and Loss: 3800.734130859375
Epoch: 3 and Loss: 3801.12841796875
Epoch: 4 and Loss: 3779.938720703125
Epoch: 5 and Loss: 3784.33984375
Epoch: 6 and Loss: 3775.806640625
Epoch: 7 and Loss: 3777.026123046875
Epoch: 8 and Loss: 3771.6162109375
Epoch: 9 and Loss: 3769.04296875
Epoch: 10 and Loss: 3763.0888671875
Epoch: 11 and Loss: 3761.559814453125
Epoch: 12 and Loss: 3759.893310546875
Epoch: 13 and Loss: 3760.466796875
Epoch: 14 and Loss: 3755.538818359375
Epoch: 15 and Loss: 3753.603271484375
Epoch: 16 and Loss: 3756.83935546875
Epoch: 17 and Loss: 3756.18115234375
Epoch: 18 and Loss: 3752.11474609375
Epoch: 19 and Loss: 3752.962158203125
Epoch: 20 and Loss: 3755.560302734375
Epoch: 21 and Loss: 3752.82373046875
Epoch: 22 and Loss: 3749.8623046875
Epoch: 23 and Loss: 3754.617431640625
Epoch: 24 and Loss: 3744.50146484375
Epoch: 25 and Loss: 3739.484619140625
Epoch: 26 and Loss: 3738.968505859375
Epoch: 27 and 

KeyboardInterrupt: ignored

In [ ]:
import torch

def prediction_accurary(time):
    accs = []

    num = 5
    for x, y in data_loader:
      x=x.to('cuda')
      y=y.to('cuda')
      random_t = torch.ones(x.shape[0], device='cuda') * time
      z = torch.randn_like(x, device = 'cuda')
      mu, std= sde.marginal(x, random_t)
      perturbed_x=mu+std*z

      #print(random_t.device, perturbed_x.device)

      pred = model(perturbed_x, random_t)

      acc = (y==torch.argmax(pred, dim = 1)).sum() / x.shape[0]
      accs+=[acc]

      if num < 25:
        break
      num-=1

    return sum(accs) / len(accs)


prediction_accurary(0.1)

tensor(1., device='cuda:0')